In [1]:
def wegmans():
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    PATH = Service("/Users/gindychen/Desktop/chromedriver_mac_arm64 (1)/chromedriver.exe")
    driver = webdriver.Chrome(service = PATH)
    driver.get("https://shop.wegmans.com/shop/categories?shopping_context=instore")

    # Select In-store and Johnson City location
    time.sleep(10)

    location_xpath = "//*[@id='sticky-react-header']/div/div[2]/div[1]/div/div[3]/button"
    link = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, location_xpath)))
    link.click()

    location_search = WebDriverWait (driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='shopping-selector-search-cities']")))
    location_search.clear()
    location_search.send_keys("Johnson City")
    location_search.send_keys(Keys.RETURN)

    johnson_xpath = "//*[@id='shopping-selector-update-home-store-45-instore']"
    link = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, johnson_xpath)))
    link.click()

    # Search for item
    search = WebDriverWait (driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='sticky-react-header']/div/div[1]/div[2]/form/div/input")))
    search.clear()
    search.send_keys(string)
    search.send_keys(Keys.RETURN)

    # Scrape first page
    time.sleep(10)
    
    try:
        main = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/main/div[2]/div[4]")))
    #     print(main.text)

        products = main.find_elements(By.CLASS_NAME, "css-h5b5ap")
        global name_list
        global price_list
        global rating_list
        global img_list
        name_list = []
        price_list = []
        aisle_list = []
        rating_list = []
        img_list = []

        num = 1

        for i in products:
            if num < 6:
                name = i.find_element(By.CLASS_NAME, "css-131yigi").text
                price = i.find_element(By.CLASS_NAME, "css-zqx11d").text
                aisle = i.find_element(By.CLASS_NAME, "css-8uhtka").text
                rating = i.find_element(By.CLASS_NAME, "css-1kj4mk8").get_attribute("aria-label")
                img = i.find_element(By.CLASS_NAME, "css-15zffbe").get_attribute("src")
                
                name_list.append(name)
                price_list.append(price[:-3])
                aisle_list.append(aisle)
                rating_list.append(rating[7:])
                img_list.append(img)

                num +=1
    finally:
        """print(name_list)
        print(price_list)
        print(rating_list)"""
#         print(img_list)
        driver.quit()

In [2]:
def amazon():
    import selenium
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.support import expected_conditions as EC
    import time
    PATH = Service("/Users/gindychen/Desktop/chromedriver_mac_arm64 (1)/chromedriver.exe")
    driver = webdriver.Chrome(service = PATH)

    driver.get("https://www.amazon.com/")
    #print(driver.title)
    search =driver.find_element(By.ID,'twotabsearchtextbox')
    search.send_keys(string)
    search.send_keys(Keys.RETURN)
    #print(driver.page_source)
    global amazon_names
    global amazon_prices
    global amazon_reviews
    global amazon_image
    amazon_names=[]
    amazon_prices=[]
    amazon_reviews=[]
    amazon_image = []
    time.sleep(5)
    all_products=driver.find_elements(By.XPATH,"//div[@data-component-type='s-search-result']")
    num = 1
    for product in all_products:
        images = product.find_element(By.CLASS_NAME,"s-image")
        source =images.get_attribute('src')
        amazon_image.append(source)
        names= product.find_element(By.XPATH,".//span[@class='a-size-base-plus a-color-base a-text-normal']")
        amazon_names.append(names.text)
        prices_whole = product.find_element(By.CLASS_NAME,"a-price-whole").text
        prices_part = product.find_element(By.CLASS_NAME,"a-price-fraction").text
        amazon_prices.append(f"{prices_whole}.{prices_part}")
        reviews=product.find_element(By.XPATH,".//span[@class='a-icon-alt']").get_attribute("textContent") 
        amazon_reviews.append(reviews[:-5])
        if num>5:
            break
        else:
            num += 1




In [3]:
from tkinter import *
import time
import tkinter 
from PIL import Image, ImageTk
from urllib.request import urlopen 

root = Tk()  #root widget # parent widget
root.title("COMPARE")
root.geometry("1300x900") #set the window size 

topframe = Frame(root)
topframe.pack()
bottomframe=Frame(root)
bottomframe.pack()


# blank=Label(topframe, text = " ", padx=100)
title = Label(topframe, text="COMPARE", bg= "lightblue", font=("Arial", 20), padx=400, pady=20, fg = "white").grid (row =0, sticky='we')
intro= "On this website, you can compare products across Wegmans and Amazon. What product do you want to search up? "
text_box = Text(topframe, height = 2, width = 97, padx=13, pady=10, font=("Arial", 15), wrap=WORD)
text_box.insert(INSERT, intro)
text_box.grid(row = 1, sticky='we')

#submit and save the product the user wants to search up
string = ""
def submit():
    global entry 
    global string
    string= entry.get()
    if string != "":
        Return_statement = Label(topframe, text = f"The product you want to search up is {string}.").grid(row =4)
    wegmans()
    left_table()
    amazon()
    right_table()
    img()
    
button = Button(topframe, text="Submit", command=submit)
button.grid(row=3)
#Get product the user wants
entry= Entry(topframe)
entry.focus_set()
entry.grid(row=2, sticky='we')
topframe.grid_columnconfigure(0, weight=1)

#wegmans
def left_table():
    table_head1 = Label(bottomframe, text="Wegmans", font=("Arial", 18) )
    table_head1.grid(row =0, column=1)
    row_num=1
    for i in range(0, len(name_list)):
        canvas1i = Canvas(bottomframe, width=325, height=120, bg="light goldenrod")
        canvas1i.grid(row=row_num, column=1)
        name_display = canvas1i.create_text(150, 50, text=f"Name : {name_list[i]}",width = 260,  justify = LEFT, fill="black", font=('Helvetica 11'))
        price_display = canvas1i.create_text(150, 100, text=f"Price : {price_list[i]}", fill="black", font=('Helvetica 11'))
        rating_display = canvas1i.create_text(150, 115, text=f"Rating : {rating_list[i]}", fill="black", font=('Helvetica 11'))
        row_num+= 1 
        
#amazon
def right_table():
    table_head2 = Label(bottomframe, text="Amazon", font=("Arial", 18))
    table_head2.grid(row =0, column=3)
    row_num=1
    for i in range(0,5):
        canvas2i = Canvas(bottomframe, width=325, height=120, bg="SkyBlue1")
        canvas2i.grid(row=row_num, column=3)
        name_display = canvas2i.create_text(150, 50, text=f"Name : {amazon_names[i]}", width = 260, fill="black", font=('Helvetica 11'))
        price_display = canvas2i.create_text(150, 100, text=f"Price : ${amazon_prices[i]}", fill="black", font=('Helvetica 11'))
        rating_display = canvas2i.create_text(150, 115, text=f"Rating : {amazon_reviews[i]}", fill="black", font=('Helvetica 11'))
        row_num += 1
        
def img():
    image_link0 = img_list[0]
    u0= urlopen(image_link0)
    raw0 = u0.read()
    u0.close()
    raw_img0 = ImageTk.PhotoImage(data=raw0)
    img0 = raw_img0._PhotoImage__photo.subsample(3)
    img_display0 = Label(bottomframe, image = img0).grid(row=1, column = 0)
    
    image_link1= img_list[1]
    u1= urlopen(image_link1)
    raw1 = u1.read()
    u1.close()
    raw_img1 = ImageTk.PhotoImage(data=raw1)
    img1 = raw_img1._PhotoImage__photo.subsample(3)
    img_display1 = Label(bottomframe, image = img1).grid(row=2, column = 0)
    
    image_link2 = img_list[2]
    u2= urlopen(image_link2)
    raw2 = u2.read()
    u2.close()
    raw_img2 = ImageTk.PhotoImage(data=raw2)
    img2 = raw_img2._PhotoImage__photo.subsample(3)
    img_display2 = Label(bottomframe, image = img2).grid(row=3, column = 0)
    
    image_link3 = img_list[3]
    u3= urlopen(image_link3)
    raw3 = u3.read()
    u3.close()
    raw_img3 = ImageTk.PhotoImage(data=raw3)
    img3 = raw_img3._PhotoImage__photo.subsample(3)
    img_display3 = Label(bottomframe, image = img3).grid(row=4, column = 0)
    
    image_link4 = img_list[4]
    u4= urlopen(image_link4)
    raw4 = u4.read()
    u4.close()
    raw_img4 = ImageTk.PhotoImage(data=raw4)
    img4 = raw_img4._PhotoImage__photo.subsample(3)
    img_display4 = Label(bottomframe, image = img4).grid(row=5, column = 0)

    image_link00 = amazon_image[0]
    u00= urlopen(image_link00)
    raw00 = u00.read()
    u00.close()
    raw_img00 = ImageTk.PhotoImage(data=raw00)
    img00 = raw_img00._PhotoImage__photo.subsample(3)
    img_display00 = Label(bottomframe, image = img00).grid(row=1, column = 2)
    
    image_link11= amazon_image[1]
    u11= urlopen(image_link11)
    raw11 = u11.read()
    u11.close()
    raw_img11 = ImageTk.PhotoImage(data=raw11)
    img11 = raw_img11._PhotoImage__photo.subsample(3)
    img_display11 = Label(bottomframe, image = img11).grid(row=2, column = 2)
    
    image_link22 = amazon_image[2]
    u22= urlopen(image_link22)
    raw22 = u22.read()
    u22.close()
    raw_img22 = ImageTk.PhotoImage(data=raw22)
    img22 = raw_img22._PhotoImage__photo.subsample(3)
    img_display22 = Label(bottomframe, image = img22).grid(row=3, column = 2)
    
    image_link33 = amazon_image[3]
    u33= urlopen(image_link33)
    raw33 = u33.read()
    u33.close()
    raw_img33 = ImageTk.PhotoImage(data=raw33)
    img33 = raw_img33._PhotoImage__photo.subsample(3)
    img_display33 = Label(bottomframe, image = img33).grid(row=4, column = 2)
    
    image_link44 = amazon_image[4]
    u44= urlopen(image_link44)
    raw44 = u44.read()
    u44.close()
    raw_img44 = ImageTk.PhotoImage(data=raw44)
    img44 = raw_img44._PhotoImage__photo.subsample(3)
    img_display44 = Label(bottomframe, image = img44).grid(row=5, column = 2)
    
    image_link5 = img_list[5]
    u5= urlopen(image_link5)
    raw5 = u5.read()
    u5.close()
    raw_img5 = ImageTk.PhotoImage(data=raw5)
    img5 = raw_img5._PhotoImage__photo.subsample(3)
    img_display5 = Label(bottomframe, image = img5).grid(row=5, column =0 )
   
    
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Julia\anaconda32\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Julia\AppData\Local\Temp\ipykernel_19376\2136697768.py", line 32, in submit
    wegmans()
  File "C:\Users\Julia\AppData\Local\Temp\ipykernel_19376\3097259500.py", line 61, in wegmans
    price = i.find_element(By.CLASS_NAME, "css-zqx11d").text
  File "C:\Users\Julia\anaconda32\lib\site-packages\selenium\webdriver\remote\webelement.py", line 425, in find_element
    return self._execute(Command.FIND_CHILD_ELEMENT, {"using": by, "value": value})["value"]
  File "C:\Users\Julia\anaconda32\lib\site-packages\selenium\webdriver\remote\webelement.py", line 403, in _execute
    return self._parent.execute(command, params)
  File "C:\Users\Julia\anaconda32\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 440, in execute
    self.error_handler.check_response(response)
  File "C:\Users\Julia